In [2]:
from IPython.display import display
import pandas as pd

In [4]:
            # read in data from MassGov source and merge iteratively #

df = pd.read_csv('../data_process/Cancer_UA_AD.csv', usecols=['Geography', 'Measure'])
df.columns = ['Geography', 'cancer_admissions_UA']

ca2 = pd.read_csv('../data_process/Cancer_AA_AD.csv', usecols=['Geography', 'Measure'])
ca2.columns = ['Geography', 'cancer_admissions_AA']

df = df.merge(ca2, how='inner', on='Geography')

ca3 = pd.read_csv('../data_process/Cancer_UA_ED.csv', usecols=['Geography', 'Measure'])
ca3.columns = ['Geography', 'cancer_emergencies_UA']

df = df.merge(ca3, how='inner', on='Geography')

ca4 = pd.read_csv('../data_process/Cancer_AA_ED.csv', usecols=['Geography', 'Measure'])
ca4.columns = ['Geography', 'cancer_emergencies_AA']

df = df.merge(ca4, how='inner', on='Geography')

cv1 = pd.read_csv('../data_process/CardioVascular_UA_AD.csv', usecols=['Geography', 'Measure'])
cv1.columns = ['Geography', 'cardio_admissions_UA']

df = df.merge(cv1, how='inner', on='Geography')

cv2 = pd.read_csv('../data_process/CardioVascular_AA_AD.csv', usecols=['Geography', 'Measure'])
cv2.columns = ['Geography', 'cardio_admissions_AA']

df = df.merge(cv2, how='inner', on='Geography')

cv3 = pd.read_csv('../data_process/CardioVascular_UA_ED.csv', usecols=['Geography', 'Measure'])
cv3.columns = ['Geography', 'cardio_emergencies_UA']

df = df.merge(cv3, how='inner', on='Geography')

cv4 = pd.read_csv('../data_process/CardioVascular_AA_ED.csv', usecols=['Geography', 'Measure'])
cv4.columns = ['Geography', 'cardio_emergencies_AA']

df = df.merge(cv4, how='inner', on='Geography')

d1 = pd.read_csv('../data_process/Diabetes_UA_AD.csv', usecols=['Geography', 'Measure'])
d1.columns = ['Geography', 'Diabetes_admissions_UA']

df = df.merge(d1, how='inner', on='Geography')

d2 = pd.read_csv('../data_process/Diabetes_AA_AD.csv', usecols=['Geography','Measure'])
d2.columns = ['Geography', 'Diabetes_admissions_AA']

df = df.merge(d2, how='inner', on='Geography')

d3 = pd.read_csv('../data_process/Diabetes_UA_ED.csv', usecols=['Geography', 'Measure'])
d3.columns = ['Geography', 'Diabetes_emergencies_UA']

df = df.merge(d3, how='inner', on='Geography')

d4 = pd.read_csv('../data_process/Diabetes_AA_ED.csv', usecols=['Geography', 'Measure'])
d4.columns = ['Geography', 'Diabetes_emergencies_AA']

df = df.merge(d4, how='inner', on='Geography')

m1 = pd.read_csv('../data_process/MentalHealth_UA_AD.csv', usecols=['Geography', 'Measure'])
m1.columns = ['Geography', 'Mental_admissions_UA']

df = df.merge(m1, how='inner', on='Geography')

m2 = pd.read_csv('../data_process/MentalHealth_AA_AD.csv', usecols=['Geography', 'Measure'])
m2.columns = ['Geography', 'Mental_admissions_AA']

df = df.merge(m2, how='inner', on='Geography')

m3 = pd.read_csv('../data_process/MentalHealth_UA_ED.csv', usecols=['Geography', 'Measure'])
m3.columns = ['Geography', 'Mental_emergencies_UA']

df = df.merge(m3, how='inner', on='Geography')

m4 = pd.read_csv('../data_process/MentalHealth_AA_ED.csv', usecols=['Geography', 'Measure'])
m4.columns = ['Geography', 'Mental_emergencies_AA']

df = df.merge(m4, how='inner', on='Geography')

s1 = pd.read_csv('../data_process/Stroke_UA_AD.csv', usecols=['Geography', 'Measure'])
s1.columns = ['Geography', 'Stroke_admissions_UA']

df = df.merge(s1, how='inner', on='Geography')

s2 = pd.read_csv('../data_process/Stroke_AA_AD.csv', usecols=['Geography', 'Measure'])
s2.columns = ['Geography', 'Stroke_admissions_AA']

df = df.merge(s2, how='inner', on='Geography')

s3 = pd.read_csv('../data_process/Stroke_UA_ED.csv', usecols=['Geography', 'Measure'])
s3.columns = ['Geography', 'Stroke_emergencies_UA']

df = df.merge(s3, how='inner', on='Geography')

s4 = pd.read_csv('../data_process/Stroke_AA_ED.csv', usecols=['Geography', 'Measure'])
s4.columns = ['Geography', 'Stroke_emergencies_AA']

df_main = df.merge(s4, how='inner', on='Geography')
df_main.set_index('Geography', inplace=True)

In [5]:
# helper function to fix location entries and filter out unneccessary rows from DOE data 
all_communities = df_main.index.tolist()
def fix_location(df, place_col, val_cols):
    for place in df[place_col]:
        if place not in all_communities:
            if '-' in place:
                first = place.split('-')[0]
                second = place.split('-')[1]
                if place=='Bristol-Plymouth Regional Vocational Technical':
                    second = 'Plymouth'
                if first in all_communities:
                    new = {place_col : first}
                    for val_col in val_cols:
                        new[val_col] = df.loc[df.index[df[place_col]==place]][val_col].values[0]
                    df = df.append(new, ignore_index=True)
                if second in all_communities:
                    new = {place_col : second}
                    for val_col in val_cols:
                        new[val_col] = df.loc[df.index[df[place_col]==place]][val_col].values[0]
                    df = df.append(new, ignore_index=True)
            df.drop(df.index[df[place_col]==place], inplace=True) 
    df.set_index(place_col, inplace=True)
    df = df.loc[~df.index.duplicated(keep='first')]
    return df

            # read in data from DOE source, fix columns and merge iteratively #
e1 = pd.read_excel('../data_process/EducatorEvalPerf.xlsx', skiprows=1, \
                  usecols=[0, 5, 6, 7, 8])
val_cols = list(e1.columns)
val_cols.remove('District Name')
e1 = fix_location(e1, 'District Name', val_cols)
df_main = df_main.join(e1, how='left')

e2 = pd.read_excel('../data_process/sat_performance.xlsx', skiprows=1,\
                  usecols=[0, 3, 4, 5])
val_cols = list(e2.columns)
val_cols.remove('District Name')
e2 = fix_location(e2, 'District Name', val_cols)
df_main = df_main.join(e2, how='left')

e3 = pd.read_excel('../data_process/dropout.xlsx', skiprows=1, usecols=[0, 4])
val_cols = list(e3.columns)
val_cols.remove('District Name')
e3 = fix_location(e3, 'District Name', val_cols)
df_main = df_main.join(e3, how='left')

e4 = pd.read_excel('../data_process/Gradsattendingcollege.xlsx', skiprows=1,\
                  usecols=[0, 4])
val_cols = list(e4.columns)
val_cols.remove('District Name')
e4 = fix_location(e4, 'District Name', val_cols)
df_main = df_main.join(e4, how='left')
display(df_main.head())
                                    # sanity check #
print(set(e1.index).issubset(set(all_communities)))
print(set(e2.index).issubset(set(all_communities)))
print(set(e3.index).issubset(set(all_communities)))
print(set(e4.index).issubset(set(all_communities)))

,cancer_admissions_UA,cancer_admissions_AA,cancer_emergencies_UA,cancer_emergencies_AA,cardio_admissions_UA,cardio_admissions_AA,cardio_emergencies_UA,cardio_emergencies_AA,Diabetes_admissions_UA,Diabetes_admissions_AA,...,Stroke_emergencies_AA,% Exemplary,% Proficient,% Needs Improvement,% Unsatisfactory,Reading / Writing,Writing,Math,% Dropout All Grades,Attending Coll./Univ. (%)
Geography,,,,,,,,,,,,,,,,,,,,,
Abington,423.9,356.4,NS,NS,1713.3,1645.6,494.6,463.4,206.1,192,...,113.7,12.7,86.0,0.7,0.7,550.0,NaN,535.0,1.2,69.9
Acton,347.4,314.2,NS,NS,933.7,893.6,443,413.8,56.5,66.6,...,NS,13.1,86.5,0.4,0.0,656.0,NaN,684.0,0.2,90.0
Acushnet,460.9,337,NS,NS,2016.5,1427.9,422.5,347.4,182.4,170.9,...,NS,7.4,84.0,8.6,0.0,NaN,NaN,NaN,NaN,NaN
Adams,429.1,324.1,0,0,1704.4,1149.8,679.4,588.9,NS,NS,...,NS,13.3,84.8,1.9,0.0,522.0,NaN,522.0,3.0,66.3
Agawam,472.9,343.6,NS,NS,2347,1487.7,685,504.7,250.4,199.6,...,NS,11.7,88.0,0.3,0.0,548.0,NaN,555.0,1.1,77.8


True
True
True
True


In [6]:
t = pd.read_csv('../data_process/tobaccoDensity.csv', \
                                            usecols=['Geography', 'Measurement'])
t.set_index('Geography', inplace=True)
t = t.rename(columns={'Measurement':'tobaccoDensity'})
df_main = df_main.join(t, how='inner')

M = pd.read_excel('../data_process/Mun_Credit_Ratings.xls', usecols=[1, 12],
                                                         names = ['Name', 'MunCreditRating'])
M = fix_location(M, 'Name', ['MunCreditRating'])
df_main = df_main.join(M, how='left')

## FINAL DATAFRAME ##
display(df_main.index)
df_main.to_csv('../data_process/subset1.csv')

Index(['Abington', 'Acton', 'Acushnet', 'Adams', 'Agawam', 'Alford',
       'Amesbury', 'Amherst', 'Andover', 'Aquinnah',
       ...
       'Wilmington', 'Winchendon', 'Winchester', 'Windsor', 'Winthrop',
       'Woburn', 'Worcester', 'Worthington', 'Wrentham', 'Yarmouth'],
      dtype='object', name='Geography', length=351)